In [ ]:
import os
import dotenv
import openai
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript, clear_output

dotenv.load_dotenv()
# Replace with your OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI API clients for both models
openai.api_key = api_key

# Function to interact with ChatGPT for GPT-4
def chat_with_gpt4(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",  # Replace with the correct model name
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ]
    )
    return response.choices[0].message["content"].strip()

# Function to interact with ChatGPT for GPT-3.5
def chat_with_gpt35(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use "gpt-3.5-turbo" for GPT-3.5 models
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ]
    )
    return response.choices[0].message["content"].strip()

# Function to handle user input and display ChatGPT responses for both models
def handle_send(sender):
    user_message = user_input.value.strip()
    
    if user_message.lower() == 'exit':
        clear_output()
        return
    
    if user_message:
        user_message_html = f'<strong class="user-message">You:</strong> {user_message}<br>'
        chat_output.value += user_message_html

        # Add loading indicators while waiting for the responses
        chat_output.value += '<div class="loading-indicator">AI (GPT-3.5) is thinking...</div>'
        chat_output.value += '<div class="loading-indicator">AI (GPT-4) is thinking...</div>'
        
        # Use callbacks to fetch and display the AI responses asynchronously
        def async_callback_gpt35(*args):
            chat_response_gpt35 = chat_with_gpt35("\n".join(conversation + [user_message]))
            chat_response_html_gpt35 = f'<strong class="ai-message">GPT-3.5:</strong> {chat_response_gpt35}<br>'
            chat_output.value += chat_response_html_gpt35
            conversation.append(chat_response_gpt35)
            chat_output.value = chat_output.value.replace('<div class="loading-indicator">AI (GPT-3.5) is thinking...</div>', '')

        def async_callback_gpt4(*args):
            chat_response_gpt4 = chat_with_gpt4("\n".join(conversation + [user_message]))
            chat_response_html_gpt4 = f'<strong class="ai-message">GPT-4:</strong> {chat_response_gpt4}<br>'
            chat_output.value += chat_response_html_gpt4
            conversation.append(chat_response_gpt4)
            chat_output.value = chat_output.value.replace('<div class="loading-indicator">AI (GPT-4) is thinking...</div>', '')

        widgets.interact(async_callback_gpt35)
        widgets.interact(async_callback_gpt4)

    user_input.value = ""

# Function to handle clearing conversation
def handle_clear(sender):
    global conversation
    conversation = []
    chat_output.value = ""

# Create widgets
chat_output = widgets.HTML()
user_input = widgets.Textarea(
    placeholder="Type your message...",
    layout=widgets.Layout(width="80%", height="100px"),  # Adjust the height
)
send_button = widgets.Button(
    description="Send",
    layout=widgets.Layout(width="10%", height="32px"),
)
clear_button = widgets.Button(
    description="Clear Conversation",
    layout=widgets.Layout(width="20%", height="32px"),
)

# Add icons to buttons
send_button.icon = "paper-plane"  # You can use other icons from FontAwesome or another source
clear_button.icon = "trash"       # You can use other icons from FontAwesome or another source

send_button.style.button_color = "#4CAF50"
send_button.style.font_weight = "bold"
clear_button.style.button_color = "#f44336"  # Red color for clearing

# Apply CSS styles for the send button
send_button.add_class("send-button")

# Style the chat output with a darker color scheme
chat_output.add_class("chat-output")

# Bind the handle_send function to the button's click event
send_button.on_click(handle_send)
clear_button.on_click(handle_clear)

# JavaScript to enable sending by pressing Enter
enter_key_handler = Javascript("""
document.querySelector(".widget-textarea").addEventListener("keydown", function(e) {
    if (e.key === "Enter" && !e.shiftKey) {
        e.preventDefault();
        document.querySelector(".send-button").click();
    }
});
""")
display(enter_key_handler)

# Start a conversation
conversation = ["Text Davinci: Hello! How can I assist you today?", "GPT-4: Hello! How can I assist you today?"]

# Create a container for the input widgets
input_container = widgets.VBox([user_input, send_button, clear_button])  # Use VBox for vertical alignment

# Display widgets with enhanced styling and layout
display(widgets.VBox([chat_output, input_container], layout=widgets.Layout(padding="20px", width="80%")))

# Apply CSS styling for a visually appealing interface
style = HTML("""
<style>
body {
    background-color: #000000; /* Light background color for the entire interface */
    color: #FFFFFF; /* White text color for the entire interface */
    font-family: Arial, sans-serif; /* Choose a readable font */
    margin: 0;
    padding: 0;
}

.chat-output {
    padding: 10px;
    background-color: #222; /* Darker background color for chat history */
    color: #FFFFFF;
    border-radius: 10px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); /* Subtle box shadow for chat history */
    overflow-y: auto;
    max-height: 300px;
}

.textarea-box textarea {
    resize: none;
    border-radius: 5px;
    border: 1px solid #ccc;
    padding: 10px;
    font-size: 16px;
    background-color: #404040; /* Darker background color for textarea */
    color: #FFFFFF; /* White text color for textarea */
    flex-grow: 1;
    margin-bottom: 10px; /* Add some space below the textarea */
}

/* Style "You" text to differentiate user messages */
.user-message {
    color: #ADD8E6; /* Light blue color for "You" text */
}

/* Style the AI's response */
.ai-message {
    color:#90EE90; /* Light green color for AI responses */
}

/* Loading indicator style */
.loading-indicator {
    text-align: center;
    padding: 10px;
    font-style: italic;
    color: #888;
}
</style>
""")
display(style)
